In [1]:
from google.colab import drive
drive.mount('/content/grdrive')

Drive already mounted at /content/grdrive; to attempt to forcibly remount, call drive.mount("/content/grdrive", force_remount=True).


In [2]:
pip uninstall tensorflow

Uninstalling tensorflow-1.13.2:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.13.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.13.2


In [3]:
pip install tensorflow==1.13.2

  Using cached https://files.pythonhosted.org/packages/db/d3/651f95288a6cd9094f7411cdd90ef12a3d01a268009e0e3cd66b5c8d65bd/tensorflow-1.13.2-cp36-cp36m-manylinux1_x86_64.whl


In [4]:
!git clone https://github.com/openai/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:
import os
os.chdir("gpt-2")

In [6]:
!pip3 install -r requirements.txt

In [7]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.27Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 71.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.35Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:15, 87.9Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 8.57Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 63.6Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 64.3Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')

In [10]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

            return interact_model

In [14]:
interact_model(
    '345M',
    None,
    1,
    1,
    25,
    1,
    0,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Today I learned
======================================== SAMPLE 1 ========================================
 that there was a podcast called 'Andy Grammer's Nirvana'.com.' I didn't really come across it from that,


<function __main__.interact_model>

In [15]:
interact_model(
    '345M',
    None,
    1,
    1,
    25,
    5,
    0,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Today I learned
======================================== SAMPLE 1 ========================================
 Moments asked Majoranders bridgesaurus300 Convertowing eve eternal devices condol675 Frequency sy voted Pegasus reinventauthent repositories Britons glassro rings


<function __main__.interact_model>

In [16]:
interact_model(
    '345M',
    None,
    1,
    1,
    25,
    10,
    0,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Today I learned
======================================== SAMPLE 1 ========================================
 outdoor 30 Killed Kon BTC Corm Binding administratorsanto Jedi signify ECBadvOSED SAR 349 accidentally UnionOUP Columbia angel patterns UE Paralleloch


<function __main__.interact_model>

In [17]:
interact_model(
    '345M',
    None,
    1,
    1,
    25,
    1,
    5,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Today I learned
======================================== SAMPLE 1 ========================================
 that my husband was going to die soon and I was going to lose him.

I was scared, and I cried


<function __main__.interact_model>

In [18]:
interact_model(
    '345M',
    None,
    1,
    1,
    25,
    1,
    10,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Today I learned
======================================== SAMPLE 1 ========================================
 that you can't get an education without having a job.

The first lesson was that education is not something you can


<function __main__.interact_model>

In [19]:
interact_model(
    '345M',
    None,
    1,
    1,
    25,
    1,
    25,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Today I learned
======================================== SAMPLE 1 ========================================
 that our current system of taxation and transfer payments isn't based upon a rational calculation of the cost of living.

In


<function __main__.interact_model>